In [62]:
import pandas_datareader.data as web
import pandas as pd
from pathlib import Path

#load daily price data and earning dates data
df_earningdate=pd.read_csv('data/sp500_earningdate.csv')

In [63]:
financial_markets = ['^GSPC','^RUT','^VIX','GC=F', 'CL=F','^TNX', '^FVX']
start_date= '2016-01-01'
end_date= '2021-12-31'

In [66]:
def data_maker(tickers, start_date, end_date):
    frames=[]
    for ticker in tickers:
        data = web.DataReader(ticker, 'yahoo', start=start_date, end=end_date)[['Close']]
        data.columns = [ticker]
        pctchange_5 = pd.Series(data[ticker].pct_change(5), name = ticker+'_pctchange_5') 
        pctchange_10 = pd.Series(data[ticker].pct_change(10), name = ticker+'_pctchange_10') 
        pctchange_30 = pd.Series(data[ticker].pct_change(10), name = ticker+'_pctchange_30') 
        data = data.join(pctchange_5).join(pctchange_10).join(pctchange_30)
        frames.append(data)
        
    return pd.concat(frames,axis=1).reset_index()

In [67]:
fin_mkts_df = data_maker(financial_markets, start_date=start_date, end_date=end_date)
fin_mkts_df[:10]

,Date,^GSPC,^GSPC_pctchange_5,^GSPC_pctchange_10,^GSPC_pctchange_30,^RUT,^RUT_pctchange_5,^RUT_pctchange_10,^RUT_pctchange_30,^VIX,...,CL=F_pctchange_10,CL=F_pctchange_30,^TNX,^TNX_pctchange_5,^TNX_pctchange_10,^TNX_pctchange_30,^FVX,^FVX_pctchange_5,^FVX_pctchange_10,^FVX_pctchange_30
0,2016-01-04,2012.660034,NaN,NaN,NaN,1108.619995,NaN,NaN,NaN,20.700001,...,NaN,NaN,2.245,NaN,NaN,NaN,1.735,NaN,NaN,NaN
1,2016-01-05,2016.709961,NaN,NaN,NaN,1110.439941,NaN,NaN,NaN,19.340000,...,NaN,NaN,2.248,NaN,NaN,NaN,1.724,NaN,NaN,NaN
2,2016-01-06,1990.260010,NaN,NaN,NaN,1094.369995,NaN,NaN,NaN,20.590000,...,NaN,NaN,2.177,NaN,NaN,NaN,1.650,NaN,NaN,NaN
3,2016-01-07,1943.089966,NaN,NaN,NaN,1064.569946,NaN,NaN,NaN,24.990000,...,NaN,NaN,2.153,NaN,NaN,NaN,1.612,NaN,NaN,NaN
4,2016-01-08,1922.030029,NaN,NaN,NaN,1046.199951,NaN,NaN,NaN,27.010000,...,NaN,NaN,2.130,NaN,NaN,NaN,1.578,NaN,NaN,NaN
5,2016-01-11,1923.670044,-0.044215,NaN,NaN,1041.900024,-0.060183,NaN,NaN,24.299999,...,NaN,NaN,2.158,-0.038753,NaN,NaN,1.576,-0.091643,NaN,NaN
6,2016-01-12,1938.680054,-0.038692,NaN,NaN,1044.699951,-0.059202,NaN,NaN,22.469999,...,NaN,NaN,2.102,-0.064947,NaN,NaN,1.537,-0.108469,NaN,NaN
7,2016-01-13,1890.280029,-0.050235,NaN,NaN,1010.190002,-0.076921,NaN,NaN,25.219999,...,NaN,NaN,2.066,-0.050988,NaN,NaN,1.509,-0.085455,NaN,NaN
8,2016-01-14,1921.839966,-0.010936,NaN,NaN,1025.670044,-0.036540,NaN,NaN,23.950001,...,NaN,NaN,2.098,-0.025546,NaN,NaN,1.519,-0.057692,NaN,NaN
9,2016-01-15,1880.329956,-0.021696,NaN,NaN,1007.719971,-0.036781,NaN,NaN,27.020000,...,NaN,NaN,2.033,-0.045540,NaN,NaN,1.453,-0.079214,NaN,NaN


In [69]:
fin_mkts_df.dropna(inplace=True)

fin_mkts_df=pd.merge(df_earningdate[['Earning Date']]
                   , fin_mkts_df
                   , how='left'
                   , left_on=['Earning Date']
                   , right_on=['Date'])

fin_mkts_df.drop(columns=['Date'],inplace=True)

fin_mkts_df.to_csv("data/fin_mkts.csv",index=False)

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat